In [1]:
train_file_path = "C:/Users/shabbir.hasan/Documents/Hasan/Anaconda_rep/Kaggle/whats-cooking/train.json"

In [2]:
import json

In [3]:
with open(train_file_path, encoding='utf-8') as data_file:
    data = json.loads(data_file.read())

In [4]:
from pandas.io.json import json_normalize

In [5]:
df = json_normalize(data)

In [6]:
df.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


#### Converting to text

In [7]:
from nltk.stem import WordNetLemmatizer
import re

In [8]:
import nltk

In [9]:
def create_long_str(inp):
    temp_str = ''
    for i in inp:
        temp_str += ' ' + (i.replace("''", '')) 
    return temp_str.strip()

In [10]:
create_long_str(df.ingredients[0])

'romaine lettuce black olives grape tomatoes garlic pepper purple onion seasoning garbanzo beans feta cheese crumbles'

In [11]:
df['ingredients_str'] = df.ingredients.apply(create_long_str)

In [12]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

In [13]:
#yeah its fancy
porter=PorterStemmer()

def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [14]:
stemSentence(df.ingredients_str[0])

'romain lettuc black oliv grape tomato garlic pepper purpl onion season garbanzo bean feta chees crumbl '

In [15]:
df['ingredients_stems'] = df.ingredients_str.apply(stemSentence)

In [16]:
df.head()

,cuisine,id,ingredients,ingredients_str,ingredients_stems
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes...",romaine lettuce black olives grape tomatoes ga...,romain lettuc black oliv grape tomato garlic p...
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g...",plain flour ground pepper salt tomatoes ground...,plain flour ground pepper salt tomato ground b...
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g...",eggs pepper salt mayonaise cooking oil green c...,egg pepper salt mayonais cook oil green chili ...
3,indian,22213,"[water, vegetable oil, wheat, salt]",water vegetable oil wheat salt,water veget oil wheat salt
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe...",black pepper shallots cornflour cayenne pepper...,black pepper shallot cornflour cayenn pepper o...


In [17]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df.ingredients_stems)
X_train_counts.shape

(39774, 2670)

In [18]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(39774, 2670)

In [19]:
X_train_tfidf.

SyntaxError: invalid syntax (<ipython-input-19-e73080e4393c>, line 1)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, df.cuisine)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()

In [ ]:
x = v.fit_transform(df['ingredients_stems'])

In [ ]:
x.toarray()

In [20]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy
# textblob, string
# from keras.preprocessing import text, sequence
# from keras import layers, models, optimizers

C:\Users\shabbir.hasan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [21]:
df.head()

,cuisine,id,ingredients,ingredients_str,ingredients_stems
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes...",romaine lettuce black olives grape tomatoes ga...,romain lettuc black oliv grape tomato garlic p...
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g...",plain flour ground pepper salt tomatoes ground...,plain flour ground pepper salt tomato ground b...
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g...",eggs pepper salt mayonaise cooking oil green c...,egg pepper salt mayonais cook oil green chili ...
3,indian,22213,"[water, vegetable oil, wheat, salt]",water vegetable oil wheat salt,water veget oil wheat salt
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe...",black pepper shallots cornflour cayenne pepper...,black pepper shallot cornflour cayenn pepper o...


In [22]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train = encoder.fit_transform(df['ingredients_stems'])
# valid_y = encoder.fit_transform(valid_y)

In [23]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(df['ingredients_stems'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='\\w{1,}', tokenizer=None,
        vocabulary=None)

In [24]:
# transform the training and validation data using count vectorizer object
train_count =  count_vect.transform(df['ingredients_stems'])
# xvalid_count =  count_vect.transform(valid_x)

In [25]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)

In [26]:
tfidf_vect.fit(df['ingredients_stems'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='\\w{1,}', tokenizer=None, use_idf=True,
        vocabulary=None)

In [27]:
train_tfidf =  tfidf_vect.transform(df['ingredients_stems'])

In [33]:
train_tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
train_tfidf.toarray()[1]

In [34]:
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix

In [53]:
X = sparse_random_matrix(100, 100, density=0.01, random_state=42)

In [63]:
svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
svd.fit(train_tfidf)  

TruncatedSVD(algorithm='randomized', n_components=50, n_iter=7,
       random_state=42, tol=0.0)

In [69]:
print(svd.explained_variance_ratio_)  

[0.01070914 0.02648742 0.02038598 0.01612431 0.01543306 0.01304989
 0.01123017 0.01042603 0.00960825 0.00888767 0.00865446 0.0083075
 0.00807561 0.00732984 0.00705091 0.00685169 0.00680985 0.0066117
 0.00628168 0.00603702 0.00576237 0.0056496  0.00552068 0.00547008
 0.00536851 0.00530525 0.0051552  0.00508204 0.00499084 0.00490452
 0.00483097 0.00470986 0.00465364 0.00454763 0.00448037 0.00436845
 0.0042576  0.00420758 0.00410704 0.00406444 0.004016   0.00392704
 0.00379239 0.00373532 0.0037115  0.00368966 0.00361945 0.00349325
 0.00346161 0.00339651]


In [70]:
print(svd.explained_variance_ratio_.sum())  

0.34863159157603196


In [71]:
print(svd.singular_values_)  

[56.35285712 31.55312067 27.4588247  24.41681727 23.89874509 22.03609867
 20.39414056 19.63586211 18.84747612 18.13161356 17.9008763  17.53677133
 17.28711343 16.4795554  16.15860412 15.9168155  15.8677573  15.63716926
 15.24394588 14.9428869  14.59469656 14.45479907 14.28709572 14.23012142
 14.09025941 14.00614746 13.80437211 13.70720228 13.59165892 13.46847098
 13.36604985 13.19710395 13.11710858 12.96559535 12.87133741 12.7089879
 12.54515766 12.47129531 12.32373249 12.2587019  12.18406748 12.05600155
 11.84296915 11.75227696 11.71352144 11.67880518 11.56803302 11.36406688
 11.31320342 11.20613507]


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer  

In [ ]:
X = vectorizer.fit_transform(train_tfidf)  